In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
import tensorflow as tf

tf.compat.v1.disable_v2_behavior()

In [11]:
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

In [21]:
from tensorflow.keras.utils import plot_model

plot_model(model, show_shapes=True, show_layer_names=True,rankdir='TB', to_file='model.png')

Output hidden; open in https://colab.research.google.com to view.

In [3]:
layer_contributions = {
    'mixed7': 0.2,
    'mixed10': 3.,
    'mixed9': 2.,
    'mixed8': 1.5,
}

In [4]:
layer_dict=dict([(layer.name, layer) for layer in model.layers])

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2: -2, 2: -2, :])) / scaling

In [5]:
dream = model.input

grads = K.gradients(loss, dream)[0]

grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실:', loss_value)
        x += step * grad_values
    return x

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import scipy
from tensorflow.keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)

def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)

def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[2], 3))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [8]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 100.

base_image_path = '/content/drive/MyDrive/Colab Notebooks/datasets/deepdream.png'

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='/content/drive/MyDrive/Colab Notebooks/final_dream.png')

처리할 이미지 크기 (176, 234)
... 0 번째 손실: 0.41739392
... 1 번째 손실: 0.5168768
... 2 번째 손실: 0.6573285
... 3 번째 손실: 0.8135439
... 4 번째 손실: 0.8969568
... 5 번째 손실: 0.95457673
... 6 번째 손실: 1.1116015
... 7 번째 손실: 1.1709328
... 8 번째 손실: 1.1878425
... 9 번째 손실: 1.3779532
... 10 번째 손실: 1.4850459
... 11 번째 손실: 1.5616449
... 12 번째 손실: 1.7747126
... 13 번째 손실: 1.8791552
... 14 번째 손실: 2.0076027
... 15 번째 손실: 2.1037474
... 16 번째 손실: 2.1524777
... 17 번째 손실: 2.165054
... 18 번째 손실: 2.4497805
... 19 번째 손실: 2.4444227
처리할 이미지 크기 (246, 328)
... 0 번째 손실: 0.6158054
... 1 번째 손실: 0.89379394
... 2 번째 손실: 1.1247163
... 3 번째 손실: 1.3196635
... 4 번째 손실: 1.3358774
... 5 번째 손실: 1.551711
... 6 번째 손실: 1.7560902
... 7 번째 손실: 1.7961695
... 8 번째 손실: 2.0154347
... 9 번째 손실: 2.1119194
... 10 번째 손실: 2.245759
... 11 번째 손실: 2.3600712
... 12 번째 손실: 2.6545851
... 13 번째 손실: 2.6980836
... 14 번째 손실: 2.8560739
... 15 번째 손실: 2.981555
... 16 번째 손실: 3.0403955
... 17 번째 손실: 3.1345963
... 18 번째 손실: 3.356639
... 19 번째 손실: 3.3584323
처리할 이미지 크기 (345, 4